In [1]:
all_files = !ls Raw/*/*txt
annotated_files = !ls Raw/*/*Annotated*txt
path_files = !ls Raw/*/*Path*txt

path_files = set(path_files)
annotated_files = set(annotated_files)
path_files -= annotated_files

all_files = set(all_files)
all_files -= path_files
all_files -= annotated_files
print(all_files)

{'Raw/Mario/mario-5-1.txt', 'Raw/Castlevania/CV_1.txt', 'Raw/Mario/SuperMarioBros2(J)-World3-3.txt', 'Raw/Metroid/metroid_5.txt', 'Raw/Mario/SuperMarioBros2(J)-World4-1.txt', 'Raw/Mario/SuperMarioBros2(J)-World6-3.txt', 'Raw/Ninja Gaiden/NG_11.txt', 'Raw/Mario/SuperMarioBros2(J)-World3-1.txt', 'Raw/Mario/SuperMarioBros2(J)-World1-3.txt', 'Raw/Ninja Gaiden/NG_2.txt', 'Raw/MegaMan/megaman_1_3.txt', 'Raw/Mario/mario-6-3.txt', 'Raw/MegaMan/megaman_1_1.txt', 'Raw/Mario/SuperMarioBros2(J)-World2-3.txt', 'Raw/Metroid/metroid_2.txt', 'Raw/Castlevania/CV_4.txt', 'Raw/MegaMan/megaman_1_8.txt', 'Raw/Castlevania/CV_2.txt', 'Raw/Mario/SuperMarioBros2(J)-World5-2.txt', 'Raw/Mario/mario-2-1.txt', 'Raw/Mario/SuperMarioBros2(J)-World2-1.txt', 'Raw/Mario/mario-3-1.txt', 'Raw/Mario/mario-3-3.txt', 'Raw/Mario/SuperMarioBros2(J)-World5-1.txt', 'Raw/Mario/mario-6-1.txt', 'Raw/Mario/SuperMarioBros2(J)-WorldB-1.txt', 'Raw/Mario/mario-1-2.txt', 'Raw/Mario/SuperMarioBros2(J)-WorldD-1.txt', 'Raw/Ninja Gaiden/NG_

In [2]:
f2path = {}
for file in all_files:
    
    header = file[:file.index('.txt')]
    file = header + '.txt'
    f2path[file] = []
    path = header + '_Annotated_Path.txt'
    print(path)
    possible_paths = [header+f'_Annotated_Path_{i}.txt' for i in range(6)]
    for p in [path] + possible_paths:
        if p in annotated_files:
            f2path[file].append(p)
print(f2path)

Raw/Mario/mario-5-1_Annotated_Path.txt
Raw/Castlevania/CV_1_Annotated_Path.txt
Raw/Mario/SuperMarioBros2(J)-World3-3_Annotated_Path.txt
Raw/Metroid/metroid_5_Annotated_Path.txt
Raw/Mario/SuperMarioBros2(J)-World4-1_Annotated_Path.txt
Raw/Mario/SuperMarioBros2(J)-World6-3_Annotated_Path.txt
Raw/Ninja Gaiden/NG_11_Annotated_Path.txt
Raw/Mario/SuperMarioBros2(J)-World3-1_Annotated_Path.txt
Raw/Mario/SuperMarioBros2(J)-World1-3_Annotated_Path.txt
Raw/Ninja Gaiden/NG_2_Annotated_Path.txt
Raw/MegaMan/megaman_1_3_Annotated_Path.txt
Raw/Mario/mario-6-3_Annotated_Path.txt
Raw/MegaMan/megaman_1_1_Annotated_Path.txt
Raw/Mario/SuperMarioBros2(J)-World2-3_Annotated_Path.txt
Raw/Metroid/metroid_2_Annotated_Path.txt
Raw/Castlevania/CV_4_Annotated_Path.txt
Raw/MegaMan/megaman_1_8_Annotated_Path.txt
Raw/Castlevania/CV_2_Annotated_Path.txt
Raw/Mario/SuperMarioBros2(J)-World5-2_Annotated_Path.txt
Raw/Mario/mario-2-1_Annotated_Path.txt
Raw/Mario/SuperMarioBros2(J)-World2-1_Annotated_Path.txt
Raw/Mario/mar

In [19]:
import json
jsons = !ls */*/*tile*json
known = set(['P','-','x','\n'])
per_game = {'Raw/Castlevania/CV_tile_spec.json':{'collectable':['H','O','1','2','3','4','5','6','7','F','#','S','G','K','R','>','0','A','U'],
                                             'powerup':['1','F','#','S','5','H','G','O','>','A','U'],
                                             'breakable':['B'],
                                             'solid':['B'],
                                             'hazard':['E'],
                                             'moving':['E'],
                                             'null':['@'],
                                             'portal':['D']},
            'Raw/Kid Icarus - current agent/KI_tile_specs.json':{},
            'Raw/Mario/SMB_tile_specs.json':{'collectable':['o','Q'],
                                         'powerup':['?'],
                                         'hazard':['E','B'],
                                         'moving':['E'],
                                         'breakable':["S"]},
            'Raw/MegaMan/MM_tile_specs.json':{'null': ['@'], 
                                          'solid': ['#', 'B', 'H', 'M', 'C', 'D'], 
                                          'breakable': ['B'], 
                                          'hazard': ['H'], 
                                          'climbable': ['|'], 
                                          'collectable': ['L', 'l', 'W', 'w', '+', '*', 'U'],
                                          'powerup': ['L', 'l', 'W', 'w', '+', '*', 'U'], 
                                          'moving': ['M', 'D'],
                                          'portal':['t']},
            'Raw/Metroid/MT_tile_specs.json':{'breakable': ['B'],
                                          'solid':['B','[',']'],
                                          'portal':['D','(',')',],
                                          'collectable': ['+'],
                                          'powerup': ['+'], 
                                          'null':['@'],
                                          'hazard':['E','[',']'],
                                          'moving':['E']},
            'Raw/Ninja Gaiden/NG_tile_specs.json':{
                                            'hazard':['K','B','E','D','C','M'],
                                            'moving':['K','B','E','D','C','M'],
                                            'collectable': ['F','%',')','*','W','A','1','+','J','T','R'],
                                            'powerup': ['F','%',')','*','W','A','1','+','J','T'], }
}


per_game_annotations = {}

for j in jsons:
    tiles = json.load(open(j))
    tile2annotation = {}
    for tile_type in tiles:        
        for tile in tiles[tile_type]:            
            if tile not in tile2annotation:
                tile2annotation[tile] = []                
            tile2annotation[tile].append(tile_type)
    for tile_type in per_game[j]:        
        for tile in per_game[j][tile_type]:            
            if tile not in tile2annotation:
                tile2annotation[tile] = []                
            tile2annotation[tile].append(tile_type)
    per_game_annotations[j] = tile2annotation
    annotated = set(tile2annotation.keys())
    for level in all_files:
        if level[:10] == j[:10]:
            with open(level) as level_file:
                level_str = level_file.read()
                level_vocab = set(level_str)
                missing = (level_vocab-annotated)-known
                if len(missing) > 0:
                    print(missing)
                    print(j,level)
                    
ignored = set(['passable'])  
combos = set()
for game in per_game_annotations:
    for tile in per_game_annotations[game]:
        affordances = tuple(sorted(set(per_game_annotations[game][tile])-ignored))
        combos.add(affordances)

common_vocab = {('breakable', 'solid'):'S', 
                ('climbable', 'solid'):'|', 
                ('climbable', 'passable','solid'):'|', 
                ('collectable',):'o', 
                ('collectable', 'powerup'):'*', 
                ('collectable', 'solid'):'Q', 
                ('hazard',):'v', 
                ('hazard', 'moving'):'e', 
                ('hazard', 'moving', 'solid'):'E', 
                ('hazard', 'moving', 'passable', 'solid'):'E', 
                ('hazard', 'null', 'solid'):'@', 
                ('hazard', 'solid'):'^', 
                ('moving', 'solid'):'#', 
                ('portal',):'$',  
                ('powerup', 'solid'):'!',
                ('solid',):'X',
                ('empty',):'-'}

path_vocab = {'S':'Ś', 'E':'Ě', '-':'+', 'e':'ě', 'X':'Ǩ', '*':'★', 
              '|':'¦', '^':'⌅', 'v':'ѷ', '$':'§', 'o':'©'}

start_vocab = {'S':'∫', 'E':'∃', '-':'∓', 'e':'∈', 'X':'∑', '*':'⋇', 
              '|':'≬', '^':'⊤', 'v':'⋎', '$':'∲', 'o':'∘'}

end_vocab = {'S':'√', 'E':'∄', '-':'∔', 'e':'∉', 'X':'∏', '*':'⊛', 
              '|':'∥', '^':'⊥', 'v':'⊻', '$':'∱', 'o':'⊜'}

{'W'}
Raw/Castlevania/CV_tile_spec.json Raw/Castlevania/CV_1.txt
{'C'}
Raw/Castlevania/CV_tile_spec.json Raw/Castlevania/CV_2.txt


In [37]:
def read_level_file(name):
    raw_level = []
    with open(name) as raw_level_file:
        for line in raw_level_file:
            raw_level.append(line.rstrip())
    return raw_level
def read_path_file(name):
    path = []
    with open(name) as raw_file:
        for line in raw_file:
            path.append(eval(line.rstrip()))
    return path

width = 32
height = 15
not_found = set()
paths_found = set()
json_heights = {
    'Raw/Castlevania/CV_tile_spec.json':12
}
!mkdir processed
!rm processed/*
counter = 0
seen = set()
y_offset = 0

common_height = 15

for file in f2path: 
    json_f = None
    for j in jsons: 
        if file[:10] == j[:10]:
            json_f = j
            break 
    raw_level = read_level_file(file)
    
    level_height = len(raw_level)   
    level_width = len(raw_level[0])
    height = json_heights.get(json_f,15)
    y_offset = 0
    if level_height < height:
        raw_level = ['-'*level_width]*(height-level_height) + raw_level
        y_offset = height-level_height
    if level_width < width:
        continue
    
    affordance_level = []
    for row in raw_level:
        row_ = []
        affordance_level.append(row_)
        for tok in row:            
            row_.append(common_vocab[tuple(sorted(set(per_game_annotations[json_f].get(tok,('empty',)))))])
    for path in f2path[file]:
        path_level = read_level_file(path)
        path = read_path_file(path.replace('Annotated_',''))
        path2step = {p:i for i,p in enumerate(path)}
        
        for yy_ in range(0,level_height,height):
            for xx_ in range(level_width-width):
                chonk = [[r for r in row[xx_:xx_+width]] for row in affordance_level[yy_:yy_+height]]  
                
                P = {p:path2step[p] for p in path2step if (p[0] >= xx_ and p[0] < xx_+width) and (p[1] >= yy_ and p[1] < yy_+height)}
                
                str_chonk = '\n'.join([''.join(row) for row in chonk])
                if '@' in str_chonk:
                    continue         
                    
                min_ = float('inf')
                max_ = -float('inf')
                
                for point in P:
                    if P[point] < min_:
                        min_ = P[point]
                        min_p = point
                    if P[point] > max_:
                        max_ = P[point]
                        max_p = point
                        
                str_chonk = '\n'.join([''.join(row) for row in chonk])
                for point in P:                    
                    p_x = point[0]-xx_
                    p_y = point[1]-yy_+y_offset
                    if point == min_p:
                        chonk[p_y][p_x] = start_vocab[chonk[p_y][p_x]]
                    elif point == max_p:
                        chonk[p_y][p_x] = end_vocab[chonk[p_y][p_x]]
                    else:
                        chonk[p_y][p_x] = path_vocab[chonk[p_y][p_x]]
                
                if len(chonk) < common_height:
                    chonk = [chonk[0] for _ in range(common_height-len(chonk))] + chonk
                str_chonk = '\n'.join([''.join(row) for row in chonk])
               
                with open(f'processed/{json_f.split("/")[1]}_{counter}.txt','w') as outfile:
                    counter += 1
                    outfile.write(str_chonk)
                       
#         for yy_ in range(0,level_height,height):
#             xx_ = 0
#             if len(raw_level[yy_:yy_+height]) < height:
#                 str_chonk ='\n'.join([row[xx_:xx_+width] for row in raw_level[yy_:yy_+height]])
#                 print(str_chonk)
#                 print(file,yy_,level_height)
#                 continue
#             for xx_ in range(level_width-width):
                
#                 str_chonk ='\n'.join([row[xx_:xx_+width] for row in raw_level[yy_:yy_+height]])
#                 path_chonk ='\n'.join([row[xx_:xx_+width] for row in path_level[yy_:yy_+height]])
#                 if '@' in str_chonk or 'x' not in path_chonk:
#                     continue
#                 rows = [row[xx_:xx_+width] for row in path_level[yy_:yy_+height]]
#                 all_ex = True
#                 for r in rows[7:11]:
#                     if r[12:16] != 'xxxx':
#                         all_ex = False
#                 if all_ex:
#                     print(path_chonk)
#                     print(path)
#                 common_chunk = []
#                 for tok,p in zip(str_chonk,path_chonk):
#                     if tok == '\n':
#                         common_chunk.append('\n')
#                     else:
#                         if tok not in per_game_annotations[json_f]:
#                             tok = '-'
#                         else:
#                             affordances = tuple(sorted(set(per_game_annotations[json_f][tok])))
#                             if affordances not in common_vocab:
#                                 print(tok,file,affordances)
#                             tok = common_vocab[affordances]
#                         if p == 'x':
#                             print(yy_,xx_)
#                             common_chunk.append(path_vocab[tok])
#                         else:    
#                             common_chunk.append(tok)
#                 common_chunk = ''.join(common_chunk)
#                 good = True
#                 for row in common_chunk.split('\n'):
#                     if '$' in row:
#                         if row.index('$') > 1 and row.index('$') <= width-1:
#                             good = False
#                 if common_chunk in seen:
#                     continue
#                 seen.add(common_chunk)
#                 if good:
#                     with open(f'processed/{j.split("/")[1]}_{counter}.txt','w') as outfile:
#                         counter += 1
#                         outfile.write(common_chunk)
#                 #print(str_chonk)
#                 #print('\n')\

mkdir: cannot create directory ‘processed’: File exists


In [5]:

common_vocab = {('breakable', 'solid'):'S', 
                ('climbable', 'solid'):'|', 
                ('climbable', 'passable','solid'):'|', 
                ('collectable',):'o', 
                ('collectable', 'powerup'):'*', 
                ('collectable', 'solid'):'Q', 
                ('hazard',):'v', 
                ('hazard', 'moving'):'e', 
                ('hazard', 'moving', 'solid'):'E', 
                ('hazard', 'moving', 'passable', 'solid'):'E', 
                ('hazard', 'null', 'solid'):'@', 
                ('hazard', 'solid'):'^', 
                ('moving', 'solid'):'#', 
                ('portal',):'$',  
                ('powerup', 'solid'):'!',
                ('solid',):'X'}
path_vocab = {'S':'Ś', 'E':'Ě', '-':'+', 'e':'ě', 'X':'Ǩ', '*':'★', 
              '|':'¦', '^':'⌅', 'v':'ѷ', '$':'§', 'o':'©'}

In [6]:
files = !ls processed/*txt

In [7]:
per_game = {}
for f in files:
    game = f.split('_')[0]
    if game not in per_game:
        per_game[game] = []
    per_game[game].append(f)


In [8]:
for game in per_game:
    print(game,len(per_game[game]))

processed/Castlevania 232
processed/Mario 6258
processed/MegaMan 1223
processed/Metroid 18100
processed/Ninja Gaiden 1648
